In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

# Ensemble Learning and Random Forests

The main idea is the following: If you aggregate the predictions of a group of predictors (such as classifiers or regressors), you'll often get a better predictions than with the best individual predictor. A group of predictor is called an *ensemble* thus this technique is called *Ensemble Learning*, and an Ensemble Learning algorithm is called an *Ensemble method*.

## Voting classifiers

A simple way to create a better classifier is to aggregate the predictions of each classifier and predict the class that gets the most votes. This majority-vote classifier is called a ***Hard Voting Classifier***. It often achieves a higher accuracy than the best classifier in the ensemble. In fact, even if each classifier is a *weak learner* (means it does slightly better than random guessing), the ensemble can still be a *strong learner*, provided there are a sufficient number of weak learners and they are sufficiently diverse.

Ensemble methods work bst when the predictors are as independent from one another possible. One way to get diverse classifiers is to train them using very different algorithms. This increases the chance that they will make very different types of errors, improving the ensemble's accuracy.

In [2]:
#Import algorithms
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

#Import dataset
from sklearn.datasets import make_moons

#Import train test split
from sklearn.model_selection import train_test_split

In [3]:
X,y = make_moons(n_samples=100000,noise=0.4)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [5]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [6]:
voting_clf = VotingClassifier(
            estimators=[("lr",log_clf),("rf",rnd_clf),("svm",svm_clf)],
            voting="hard",
            n_jobs=-1)

In [7]:
voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr',
                              LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=100,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                             

In [8]:
#import accuracy
from sklearn.metrics import accuracy_score

In [9]:
for clf in (log_clf,rnd_clf,svm_clf,voting_clf):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__,accuracy_score(y_test,y_pred))

LogisticRegression 0.8273939393939393
RandomForestClassifier 0.8439090909090909
SVC 0.8596969696969697
VotingClassifier 0.8554848484848485


If all the classifiers are able to estimate class probabilities, then it is possible to predict the class with the highest class probability, averaged over all the individual classifiers. This is called ***Soft voting***. It often achieves higher performance than hard voting because it gives more weight to highly confident votes. For this, just remplacing voting="soft" and ensure that every classifiers can compute class probabilities.

## Bagging and Pasting

Another approach is to use the same training algorithm for every predictor and train them on different subsets of the training set. When sampling is performed with replacement, this method is called bagging and without it is called pasting.
They both allow training instance to be sample several times across multiple predictors, but only bagging allows training instances to be sampled several times for the same predictor.

Once all predictors are trained, the ensemble can make a prediction for a new instance by simply aggregating the predictions of all predictors. The aggregation function is typically the statisical mode for classification and the average for regression. Each individual predictor has a higher bias than if it were trained on the original training set, but aggregation reducs both bias and variance. Generally, the net result is that the ensemble has a similar bias but a lower variance than a single predictor trained on the original training set.

In [10]:
#Import algorithms
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [11]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(),n_estimators=500,
        max_samples=100,bootstrap=True,n_jobs=-1)

In [12]:
bag_clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [13]:
y_pred = bag_clf.predict(X_test)

In [14]:
accuracy_score(y_test,y_pred)

0.8602121212121212

The ensemble has a comparable bias but a smaller variance (it makes roughly the same number of errors on the training set, but the decision boundary is less irregular)

Bootstrapping introduces a bit more diversity in the subsets that each predictor is trained on, so bagging ends up with a slightly higher bias than pasting, but the extra diversity means also that the predictors end up being less correlated, so the ensemble's variance is reduced.

## Out-of-Bag evaluation

With bagging, some instances may not be sampled several times for any given predictor, while others may not be sampled at all. Only about 63%of the training instances are sampled on average for each predictor. The remaining 37% of the training instances that arenot sampled are called *out-of-bag* (oob) instances. They are not the same for all the predictors. Since a predictor never sees the oob instances during training, it can be evaluated on these instances, without the need for a separate validation set. The ensemble can be evaluated by averaging the oob evaluations of each predictor. In Sklearn, oob_score=True to request an automatic oob evaluation after training.

In [15]:
bag_clf = BaggingClassifier(
        DecisionTreeClassifier(),
        bootstrap=True,
        n_estimators=500,
        n_jobs=-1,
        oob_score=True)

In [16]:
bag_clf.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                        class_weight=None,
                                                        criterion='gini',
                                                        max_depth=None,
                                                        max_features=None,
                                                        max_leaf_nodes=None,
                                                        min_impurity_decrease=0.0,
                                                        min_impurity_split=None,
                                                        min_samples_leaf=1,
                                                        min_samples_split=2,
                                                        min_weight_fraction_leaf=0.0,
                                                        presort='deprecated',
                                                        random_state=None,


In [17]:
bag_clf.oob_score_

0.8416865671641791

In [18]:
y_pred = bag_clf.predict(X_test)
accuracy_score(y_test,y_pred)

0.8416363636363636

## Random Patches and Random Subspaces

Features can be sampled as well. The result is each predictor being trained on a random subset of the input features. This technique is particulary useful when dealing with high-dimensional inputs. This is called *Random Patches method*. Keeping all training instances but sampling features is called the *Random Subspaces method*. Sampling features results in even more predictor diversity, trading a bit more bias for a lower variance.

## Random Forest

A random forest is an ensemble of Decision Tree, generally trained via the bagging method, with typically the max_sample set to the size of the training set.

In [19]:
from sklearn.ensemble import RandomForestClassifier

In [20]:
rnd_clf = RandomForestClassifier(n_estimators=500,max_leaf_nodes=16,n_jobs=-1)
rnd_clf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=16, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [21]:
y_pred_rf = rnd_clf.predict(X_test)

In [22]:
accuracy_score(y_test,y_pred_rf)

0.8578787878787879

The RandomForest introduces extra randomness when growing trees. Instead of searching for the very best feature when splitting a node, it searches for the best feature among a subset of features. This results in geater tree diversity, which trade a higher bias for a lower variance, generally yielding an overall better model.

In [23]:
#Equivalent to a RandomForest

bag_clf = BaggingClassifier(
DecisionTreeClassifier(splitter="random",max_leaf_nodes=16),n_estimators=500,max_samples=1.0,bootstrap=True,n_jobs=-1)

## Extra-Trees

It is possible to take trees even more random by also using random thresholds for each features rather than searching for the best possible thresholds ( like Decision Trees do).

A forest of such extremely random trees is called an Extremely Randomized Trees ensemble (extra-trees). This technique trades more bias for a lower variance. It also makes the Extra-Trees much faster to train than regular Random Forests, because finding the best possible threshold for each feature at every node is one of the most time-consuming tasks of growning a tree.

## Feature Importance

Another quality of Random Forests is that they make it easy to measure the relative importance of each feature. Scikit-Learn measures a feature's importance by looking at how much the tree nodes that use that feature reduce impurity on average. More precisely, it is a weighted average, where each node's weight is equal to the number of training samples that are associated with it.

In [24]:
from sklearn.datasets import load_iris

In [25]:
iris = load_iris()

In [26]:
rnd_clf = RandomForestClassifier(n_estimators=500,n_jobs=-1)
rnd_clf.fit(iris["data"],iris["target"])

for name, score in zip(iris["feature_names"],rnd_clf.feature_importances_):
    print(name,score)

sepal length (cm) 0.10253848154550228
sepal width (cm) 0.023082319827996103
petal length (cm) 0.4362213442648789
petal width (cm) 0.43815785436162263


Random Forests are very handy to get a quick understanding of what features actually matter, in particular if you need to perform feature selection.

## Boosting

Boosting (originally called hypothesis boosting) refers to any ensemble method that can combine several weak learners into a strong learner. The general idea of most boosting methods is to train predictors sequentially, each trying to correct its predecessor.

## Adaboost

When training an AdaBoost Classifier, the algorithm trains a base classifier (such as a DecisionTree), and uses it to make predictions on the training set. The algorithm then increases the relative weight of misclassified training instances. Then it trains a second classifier, using the updated weights, and again makes predictions on the training set, updates the weights, and so on.

This sequential learning technique has some similarities with Gradient Descent, except that instead of tweaking a single predictor's parameters to minimize a cost function, AdaBoost adds predictors to the ensemble, gradually make it better.

Once all predictors are trained, the ensemble makes predictions very much like bagging or pasting, except that predictors have different weights depending on their overall accuracy on the weighted training set.

/!\ There's one important drawback to this sequential learning technique: it cannot be parallelized (or only partially), since each predictor can only be trained after the previous predictor has been trained and evaluated. As a result, it does not scale as well as bagging or pasting.

A first predictor is trained and, its weighted error rate is computed on the training set.

The predictor's weight is then computed with the learning rate hyperparameter. The more the accurate the predictor is, the higher its weight will be. If it is just random guessing randomly, then its weight will be close to zero. However, if it most often wrong (less accurate than random guessing) then its weight will be negative.

Next AdaBoost updates the instance weights which boosts the weights of the misclassified instances.

Then all the instance weights are normalized.

Finally, a new predictor is trained using the updated weights, and the whole process is repeated (the new predictor's weight is computed, the instance weights are updated, then another predictor is trained, and so on). The algorithm stops when the desired number of predictors is reached, or when a perfect predictor is found.

To make prediction, AdaBoost simply computes the predictions of all the predictors and weights them using the predictor weights alphaj. The predicted class is the one that receives the majority of the weighted votes.

The following code trains an AdaBoost classifier based on 200 Decision Stumps (a Decision Tree with a single decision node and two leafs (max_depth =1 ))

In [28]:
from sklearn.ensemble import AdaBoostClassifier

In [29]:
ada_clf = AdaBoostClassifier(
        DecisionTreeClassifier(max_depth=1),
        n_estimators=200,
        algorithm="SAMME.R", #Multiclass version of Adaboost, and R stands for Real, and allows to compute class probabilities.
        learning_rate=0.5)

In [30]:
ada_clf.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [32]:
y_pred_ada = ada_clf.predict(X_test)
accuracy_score(y_test,y_pred_ada)

0.8582424242424243

If your AdaBoost ensemble is overfitting the training set, you can try reducing the number of estimators or more strongly regularizing the base estimator.

## Gradient Boosting

Just like AdaBoost, Gradient Boosting works by sequentially adding predictors to an ensemble, each one correcting its predecessor. However, instead of tweaking the instance weights at every iteration like AdaBoost does, this method tries to fit the new predictor to the residual errors made by the previous predictor.

In [33]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

#We train a second decisiontreeregressor on the residual errors made by the first predictor
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

#Then we train a third regressor on the residual errors made by the second predictor
y3 = y2 - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

In [35]:
#y_pred = sum(tree.predict(X_new) for tree in (tree_reg1,tree_reg2,tree_reg3)) TO PREDICT THE NEW INSTANCES

In [36]:
#Simpler way to do it

from sklearn.ensemble import GradientBoostingRegressor

In [37]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=1.0, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=3,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

The learning rate hyperparameter scales the contribution of each tree. If you set it to a low value, such as 0.1, you will need more trees in the ensemble to fit the training set, but the predictions will usually generalize better. This is a regularization technique called *shrinkage*.
In order to find the optimal number of threes, you can use early stopping. A simple way to implement this is to use the staged_predict() method: it returns an iterator over the predictions made by the ensemble at each stage of training (with one tree, two trees,etc.).

In [38]:
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [39]:
X_train,X_val,y_train,y_val = train_test_split(X,y)

In [40]:
gbrt = GradientBoostingRegressor(max_depth=2,n_estimators=120)
gbrt.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=120,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [42]:
errors = [mean_squared_error(y_val,y_pred) for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators = np.argmin(errors) + 1

In [44]:
gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)
gbrt.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=2,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=120,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

It is also possible to implement early stopping by actually stopping the training early by setting warm_start = True in scikit-learn (which allows increamental training by keeping existing trees).

In [48]:
gbrt = GradientBoostingRegressor(max_depth=2,warm_start=True)

min_val_error = float("inf")
error_going_up = 0

for n_estimators in range(1,120):
    gbrt.n_estimators = n_estimators
    gbrt.fit(X_train,y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val,y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up = 0 
    else:
        error_going_up += 1
        if error_going_up == 5:
            break #Early stopping

It also supports a subsample parameter, which specifies the fraction of training intances to be used for training each tree (0.25 equal 25% of the training instance selected randomly). This technique trades a higher bias against a lower variance and speeds up training considerably. This is called **Stochastic Gradient Boosting**.

The other cost functions that can be used with Gradient Boosting are controlled by the loss hyperparameter.

In [52]:
import xgboost

In [53]:
xgb_reg = xgboost.XGBRegressor()

In [54]:
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_val)

In [55]:
mean_squared_error(y_val,y_pred)

0.10186200753973923

In [56]:
xgb_reg.fit(X_train,y_train,
            eval_set=[(X_val,y_val)], early_stopping_rounds=2)

[0]	validation_0-rmse:0.41831
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.37113
[2]	validation_0-rmse:0.34520
[3]	validation_0-rmse:0.33168
[4]	validation_0-rmse:0.32473
[5]	validation_0-rmse:0.32121
[6]	validation_0-rmse:0.31941
[7]	validation_0-rmse:0.31853
[8]	validation_0-rmse:0.31810
[9]	validation_0-rmse:0.31792
[10]	validation_0-rmse:0.31784
[11]	validation_0-rmse:0.31781
[12]	validation_0-rmse:0.31775
[13]	validation_0-rmse:0.31782
[14]	validation_0-rmse:0.31785
Stopping. Best iteration:
[12]	validation_0-rmse:0.31775



XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [58]:
mean_squared_error(y_val,xgb_reg.predict(X_val))

0.10096525562839108

## Stacking

Stacking is based on a simple idea: instead of using trivial functions (like hard-voting) to aggregate the predictions of all predictors in an ensemble, why don't we train a model to perfom this aggregation?

To train the blender (or meta-learner, the final predictor), a common approach is to use a hold-out set.


First, the training set is split into two subsets. The first subsets is used to train the predictors in the first layer. Next, the first layer's predictors are used to make predictions on the second (held-out) set. This ensure that the predictions are "clean", since the predictors never saw these instances during training. For each instance in the hold-out set, there are three predicted values. We can create a new training set using these predicted values as input features (new training set 3D), and keeping the target values. The blender is trained on this new training set, so it learns to predict the target value, given the first layer's predictions.

It is actually possible to train different blenders this way to get a whole layer of blenders. The trick is to split the training set into three subsets: the first one is used to train the second layer (using predictions made by the predictors of the first layer), and the third one is used to create the training set to train the third layer (using predictions made by the second layer). Once this is done, we can make a prediction for a new instance by going through each layer sequentially.

Unfortunately, Scikit-learn does not support stacking directly (but other open-source implementation like DESlib can be used).